In [2]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import matplotlib.pyplot as plt
import math

In [3]:
df = pd.read_csv('/Users/mac/Desktop/Comapny2/CryptoPortfolioAI/artifacts/df_last_year_20250109.csv')
df.head()

,datetime,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,circulating_supply,total_supply,date
0,2025-01-09 00:38:38,WAI,0.020860,1.020250e+06,0.0000,7.788221,12.335881,12.335881,12.335881,12.335881,12.335881,0.000000e+00,0.0000,2.086005e+07,0.000000e+00,1.000000e+09,2022-02-24
1,2025-01-09 00:38:38,ADA,0.924652,1.939491e+09,-6.0118,-2.424521,-8.537684,1.392174,-16.965305,110.523816,175.285419,3.250260e+10,0.9853,4.160936e+10,3.515115e+10,4.499512e+10,2022-02-24
2,2025-01-09 00:38:38,AVAX,37.299335,7.347442e+08,7.7448,-0.934974,-7.285825,1.592909,-23.843868,28.784312,46.534919,1.531036e+10,0.4641,2.669695e+10,4.104728e+08,4.488091e+08,2022-02-24
3,2025-01-09 00:38:38,BNB,691.104599,2.295891e+09,1.4070,-0.271765,-1.152902,-1.939567,-3.938155,12.218398,23.680743,9.952353e+10,3.0170,9.952353e+10,1.440065e+08,1.440065e+08,2022-02-24
4,2025-01-09 00:38:38,BTC,94469.951290,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,1.980741e+07,1.980741e+07,2022-02-24


In [4]:
df.tail()

,datetime,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,circulating_supply,total_supply,date
13528,2025-01-08 22:57:20,DOGEVERSE,0.000010,5.188921e+03,127.3776,-1.896835,-4.512807,-3.020233,-28.189798,-49.020244,-56.165204,0.000000e+00,0.0000,1.912198e+06,0.000000e+00,2.000000e+11,2025-01-08
13529,2025-01-08 22:57:20,SEAL,0.350576,7.102184e+05,-15.6281,-5.330465,-18.265698,-24.304347,-36.906949,50.223078,3.172062,0.000000e+00,0.0000,7.362094e+06,0.000000e+00,2.100000e+07,2025-01-08
13530,2025-01-08 22:57:20,WAI,0.019406,1.061065e+06,0.0000,0.293143,4.506117,4.506117,4.506117,4.506117,4.506117,0.000000e+00,0.0000,1.940612e+07,0.000000e+00,1.000000e+09,2025-01-08
13531,2025-01-08 22:57:20,MATIC,0.451305,5.521184e+06,8.8961,-1.090564,-5.768349,-2.881374,-29.384462,19.969871,22.530613,8.637006e+08,0.0265,4.513052e+09,1.913784e+09,1.000000e+10,2025-01-08
13532,2025-01-08 22:57:20,ADA,0.940861,1.884548e+09,-5.1133,-1.415288,-8.646028,6.201881,-16.309263,114.248037,180.772359,3.307237e+10,1.0147,4.233876e+10,3.515115e+10,4.499512e+10,2025-01-08


In [7]:
df_btc = df[df["symbol"] == "BTC"]
df_btc.head()

,datetime,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,circulating_supply,total_supply,date
4,2025-01-09 00:38:38,BTC,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0,2022-02-24
18,2025-01-09 00:38:34,BTC,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0,2022-02-25
30,2025-01-09 00:38:33,BTC,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0,2022-02-26
43,2025-01-09 00:38:30,BTC,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0,2022-02-27
55,2025-01-09 00:38:28,BTC,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0,2022-02-28


In [8]:
df_btc.drop(columns=["symbol", "datetime"], inplace=True)

/var/folders/j6/9x_46crd51ld6f4qmwpg0zgm0000gn/T/ipykernel_44141/475833322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_btc.drop(columns=["symbol", "datetime"], inplace=True)


In [11]:
df_btc.rename(columns={"date": "ds"}, inplace=True)
df_btc.set_index("ds", inplace=True)

/var/folders/j6/9x_46crd51ld6f4qmwpg0zgm0000gn/T/ipykernel_44141/1378110482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_btc.rename(columns={"date": "ds"}, inplace=True)


In [12]:
df_btc

,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,circulating_supply,total_supply
ds,,,,,,,,,,,,,,
2022-02-24,94469.951290,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
2022-02-25,94469.951290,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
2022-02-26,94469.951290,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
2022-02-27,94469.951290,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
2022-02-28,94469.951290,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-04,93822.475106,6.457933e+10,20.8796,-0.831876,-3.428913,-0.142860,-3.914102,23.123302,57.587297,1.858377e+12,56.9542,1.970272e+12,19807378.0,19807378.0
2025-01-05,93822.475106,6.457933e+10,20.8796,-0.831876,-3.428913,-0.142860,-3.914102,23.123302,57.587297,1.858377e+12,56.9542,1.970272e+12,19807378.0,19807378.0
2025-01-06,93822.475106,6.457933e+10,20.8796,-0.831876,-3.428913,-0.142860,-3.914102,23.123302,57.587297,1.858377e+12,56.9542,1.970272e+12,19807378.0,19807378.0


In [19]:
# df_btc.to_csv('/Users/mac/Desktop/Comapny2/CryptoPortfolioAI/artifacts/BTC_1050days.csv')

In [3]:
newdf = pd.read_csv('/Users/mac/Desktop/Comapny2/CryptoPortfolioAI/artifacts/BTC_1050days.csv')
newdf.head()

,ds,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,circulating_supply,total_supply
0,2022-02-24,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
1,2022-02-25,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
2,2022-02-26,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
3,2022-02-27,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0
4,2022-02-28,94469.95129,6.616604e+10,20.0372,-0.083482,-2.323822,0.338016,-2.981795,23.969996,58.575804,1.871205e+12,56.6557,1.983869e+12,19807409.0,19807409.0


In [4]:
newdf.columns

Index(['ds', 'price', 'volume_24h', 'volume_change_24h', 'percent_change_1h',
       'percent_change_24h', 'percent_change_7d', 'percent_change_30d',
       'percent_change_60d', 'percent_change_90d', 'market_cap',
       'market_cap_dominance', 'fully_diluted_market_cap',
       'circulating_supply', 'total_supply'],
      dtype='object')

In [5]:
newdf.dtypes

ds                           object
price                       float64
volume_24h                  float64
volume_change_24h           float64
percent_change_1h           float64
percent_change_24h          float64
percent_change_7d           float64
percent_change_30d          float64
percent_change_60d          float64
percent_change_90d          float64
market_cap                  float64
market_cap_dominance        float64
fully_diluted_market_cap    float64
circulating_supply          float64
total_supply                float64
dtype: object

In [6]:
import plotly.express as px

newdf['ds'] = pd.to_datetime(newdf['ds'])
fig = px.line(newdf, x='ds', y='price', 
              labels={'price': 'Amount', 'ds': 'Date'}, 
              title='Line Chart of Price Over Time')


fig.show()


# Forecasting of the Binance

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import MSTL
from statsmodels.tsa.seasonal import DecomposeResult
import math
import seaborn as sns
import matplotlib.dates as mdates
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import t
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
import warnings
warnings.filterwarnings("ignore")

In [2]:
from prophet import Prophet # type: ignore
from datetime import timedelta

In [ ]:
df = df.re

In [ ]:
df_btc['ds'] = pd.to_datetime(df_btc['ds'])  
last_date = df_btc['ds'].max()
one_year_ago = last_date - timedelta(days=900)
df_btc_one_year = df_btc[df_btc['ds'] >= one_year_ago]

daily_model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True, seasonality_mode='multiplicative')
daily_model.fit(df_btc_one_year)

future_dates = daily_model.make_future_dataframe(periods=180, freq='D')  # 6 months = 180 days

forecast = daily_model.predict(future_dates)

fig = daily_model.plot(forecast)
